In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

sys.path.insert(0, '../libraries')
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/aquabyte-images/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs")

### #0. Get the datasets ready

In [ ]:
from pycocotools.coco import COCO
import matplotlib.patches as patches
import json

In [ ]:
dataset_train  = coco.CocoDataset()
dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-07.json', class_ids=[0])
dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-08.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-10.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-11.json', class_ids=[0])
dataset_train.prepare()

In [ ]:
random_image_id = np.random.choice(dataset_train.image_ids)
image = dataset_train.load_image(random_image_id)
mask = dataset_train.load_mask(random_image_id)

In [ ]:
f ,ax = plt.subplots(1, figsize=(20, 20))
ax.imshow(image); 
for k in range(mask[0].shape[-1]):
    # draw mask
    m = mask[0][...,k]
    x, y = np.nonzero(m)
    img = np.ones( (m.shape[0], m.shape[1], 3) )
    color_mask = np.random.random((1, 3)).tolist()[0]
    for i in range(3):
        img[:,:,i] = color_mask[i]
    ax.imshow(np.dstack( (img, m*0.3) ))
    
    # draw bbox
    bbox = [min(y), min(x), max(y)-min(y), max(x)-min(x)]
    rec = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], 
                            edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
    ax.add_patch(rec)
    
ax.axis('off')
plt.show()

### #1. Config

In [ ]:
image_size = 1024
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    NAME = "shapes"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 400

    VALIDATION_STEPS = STEPS_PER_EPOCH / 20
    USE_MINI_MASK= False
config = FishConfig()
config.display()

### #2. Data generator

In [ ]:
import imgaug
from imgaug import augmenters as iaa
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness
)

In [ ]:
augmentation = {'albumentations': [
    OneOf([RandomSizedCrop(min_max_height=(50, 101), height=512, width=512, p=0.5),
          PadIfNeeded(min_height=512, min_width=512, p=0.5)], p=1),    
    VerticalFlip(p=0.5),              
    RandomRotate90(p=0.5),
    OneOf([
        ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        GridDistortion(p=0.5),
        OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)                  
        ], p=0.8),
    CLAHE(p=0.8),
    RandomContrast(p=0.8),
    RandomBrightness(p=0.8),
    RandomGamma(p=0.8)], 'imgaug': None}

## 1 - Contrast

Without

In [ ]:
augmentation = {'albumentations': [   
    RandomContrast(p=0.0),
    CLAHE(p=0.0),
], 'imgaug': None}

In [ ]:
image, image_meta, gt_class_ids, gt_boxes, gt_masks = modellib.load_image_gt(dataset_train, config, 1, \
                  augmentation=augmentation, use_mini_mask=config.USE_MINI_MASK)
plt.imshow(image)
plt.show()
plt.imshow(gt_masks.squeeze())
plt.show()

With 

In [ ]:
augmentation = {'albumentations': [   
    RandomContrast(p=0.6),
    CLAHE(p=0.6),
], 'imgaug': None}

In [ ]:
image, image_meta, gt_class_ids, gt_boxes, gt_masks = modellib.load_image_gt(dataset_train, config, 1, \
                  augmentation=augmentation, use_mini_mask=config.USE_MINI_MASK)
plt.imshow(image)
plt.show()
plt.imshow(gt_masks.squeeze())
plt.show()

## 2 - Random crops

In [ ]:
augmentation = {'albumentations': [   
    OneOf([RandomSizedCrop(min_max_height=(300, 300), height=1024, width=1024, p=1.0),
          PadIfNeeded(min_height=1024, min_width=1024, p=0.5)], p=1.0)], 'imgaug': None}

In [ ]:
image, image_meta, gt_class_ids, gt_boxes, gt_masks = modellib.load_image_gt(dataset_train, config, 30, \
                  augmentation=augmentation, use_mini_mask=config.USE_MINI_MASK)
plt.imshow(image)
plt.show()
plt.imshow(gt_masks[..., 0])
plt.show()

## 3 - Brightness

In [ ]:
augmentation = {'albumentations': [   
   RandomBrightness(p=1.0),
    RandomGamma(p=1.0)], 'imgaug': None}

In [ ]:
image, image_meta, gt_class_ids, gt_boxes, gt_masks = modellib.load_image_gt(dataset_train, config, 1, \
                  augmentation=augmentation, use_mini_mask=config.USE_MINI_MASK)
plt.imshow(image)
plt.show()
plt.imshow(gt_masks[..., 0])
plt.show()

In [ ]:
augmentation = {'albumentations': [   
   OneOf([
        OpticalDistortion(p=1, distort_limit=0.6, shift_limit=0.2)                  
        ], p=1),], 'imgaug': None}

In [ ]:
image, image_meta, gt_class_ids, gt_boxes, gt_masks = modellib.load_image_gt(dataset_train, config, 1, \
                  augmentation=augmentation, use_mini_mask=config.USE_MINI_MASK)
plt.imshow(image)
plt.show()
plt.imshow(gt_masks[..., 0])
plt.show()

Since datas are rectified, not really relevant. We will start with basic things

In [ ]:
augmentation = {'albumentations': [
    OneOf([RandomSizedCrop(min_max_height=(300, 300), height=1024, width=1024, p=0.5),
          PadIfNeeded(min_height=1024, min_width=1024, p=0.5)], p=1),
    CLAHE(p=0.6),
    RandomContrast(p=0.6),
    RandomBrightness(p=0.8),
    RandomGamma(p=0.8)], 'imgaug': None}